In [16]:
import uuid

from src.code.code.init_neural import device
!pip install kagglehub[pandas-datasets]


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [21]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import os
ROOT_DIR = '/workspace/NN'
os.chdir(ROOT_DIR)

import shutil
import kagglehub
import torch

dataset_path = os.path.join(ROOT_DIR, 'neural', 'datasets', 'fashionmnist')
logs_path = os.path.join(ROOT_DIR, 'neural', 'logs')
csv_file = os.path.join(dataset_path, 'fashion-mnist_train.csv')
prod_weights_path = os.path.join(ROOT_DIR, 'neural', 'weights', 'prod')
test_weights_path = os.path.join(ROOT_DIR, 'neural', 'weights', 'lab_1')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
if not os.path.isdir(dataset_path):
    source_dir = kagglehub.dataset_download("zalando-research/fashionmnist")
    target_dir = dataset_path
    shutil.move(source_dir, target_dir)

In [19]:
import pandas as pd

data = pd.read_csv(csv_file)
data

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class FashionMNISTDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (str): Путь к CSV-файлу с данными.
            transform (callable, optional): Трансформации для изображения.
        """
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Получаем строку с данными
        sample = self.data.iloc[idx]

        # Предполагается, что в CSV первая колонка — метка,
        # а остальные 28*28 пикселей записаны последовательно.
        label = int(sample.iloc[0])
        img_array = sample.iloc[1:].values.astype(np.uint8).reshape(28, 28)

        # Если трансформации заданы, применяем их.
        if self.transform:
            img = self.transform(img_array)
        else:
            # Преобразуем в тензор, добавляя размер канала (1, 28, 28)
            img = torch.tensor(img_array, dtype=torch.float32).unsqueeze(0) / 255.0

        return img, label

'''
0 Футболка/топ
1 Брюки
2 Пуловер
3 Платье
4 Слой
5 Сандалий
6 Рубашка
7 Кроссовок
8 Мешков
9 Ботинок на щиколотке
'''

# Пример использования:
if __name__ == "__main__":
    # Определяем трансформации: конвертация numpy массива в PIL Image и затем в тензор
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor()
    ])

    # Путь к CSV-файлу (отредактируй путь в соответствии с расположением файла)
    train_csv = csv_file

    # Создаём объект датасета
    train_dataset = FashionMNISTDataset(csv_file=train_csv, transform=transform)

    # Создаём DataLoader
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)

    # Пример итерации по DataLoader'у
    for images, labels in train_loader:
        print(f"Batch images shape: {images.shape}, labels shape: {labels.shape}")
        # Здесь можно добавить код для обучения модели
        break


Batch images shape: torch.Size([64, 1, 28, 28]), labels shape: torch.Size([64])


In [14]:
from pytorch_lightning.callbacks import ModelCheckpoint
from torchmetrics.classification import MulticlassF1Score
import uuid
from datetime import datetime
import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import FashionMNIST

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

import torchmetrics

# Определяем лёгкую сверточную сеть для FashionMNIST
class LightweightFashionMNIST(nn.Module):
    def __init__(self):
        super(LightweightFashionMNIST, self).__init__()
        # Входное изображение: 1 x 28 x 28
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)   # -> 16 x 28 x 28
        self.pool  = nn.MaxPool2d(2, 2)                             # -> 16 x 14 x 14
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)    # -> 32 x 14 x 14
        # После второго pooling: 32 x 7 x 7
        self.fc1   = nn.Linear(32 * 7 * 7, 64)
        self.fc2   = nn.Linear(64, 10)  # 10 классов

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Определяем LightningModule
class FashionMNISTLitModel(pl.LightningModule):
    def __init__(self, learning_rate=1e-3):
        super(FashionMNISTLitModel, self).__init__()
        self.model = LightweightFashionMNIST()
        self.learning_rate = learning_rate

        # Метрики для обучения и валидации
        self.train_accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=10)
        self.val_accuracy = torchmetrics.Accuracy(task='multiclass',num_classes=10)
        self.val_precision = torchmetrics.Precision(task='multiclass',num_classes=10, average='macro')
        self.val_recall = torchmetrics.Recall(task='multiclass', num_classes=10, average='macro')
        self.val_auroc = torchmetrics.AUROC(task='multiclass',num_classes=10)
        self.confmat = torchmetrics.ConfusionMatrix(task='multiclass', num_classes=10)
        self.val_f1 = MulticlassF1Score( num_classes=10, average='macro')  # Добавлена метрика F1


    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.train_accuracy.update(preds, y)
        self.log('train/loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    # Вместо training_epoch_end, используем on_train_epoch_end (без аргументов)
    def on_train_epoch_end(self):
        epoch_time = time.time() - self.epoch_start_time
        self.log('train/epoch_time', epoch_time)
        acc = self.train_accuracy.compute()
        self.log('train/accuracy', acc, prog_bar=True)
        self.train_accuracy.reset()
        self.log('train/lr', self.learning_rate)

    def on_train_epoch_start(self):
        self.epoch_start_time = time.time()

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.val_accuracy.update(preds, y)
        self.val_precision.update(preds, y)
        self.val_recall.update(preds, y)
        self.val_auroc.update(F.softmax(logits, dim=1), y)
        self.val_f1.update(preds, y)  # Обновление F1 метрики

        self.confmat.update(preds, y)
        self.log('val/loss', loss, prog_bar=True, on_epoch=True)
        return loss

    def on_validation_epoch_end(self):
        acc = self.val_accuracy.compute()
        prec = self.val_precision.compute()
        rec = self.val_recall.compute()
        f1 = self.val_f1.compute()  # Вычисление F1 метрики
        auroc = self.val_auroc.compute()
        self.log('val/accuracy', acc, prog_bar=True)
        self.log('val/precision', prec)
        self.log('val/recall', rec)
        self.log('val/auroc', auroc)
        self.log('val/f1', f1)  # Логирование F1 метрики

        # Логируем матрицу ошибок как изображение в TensorBoard
        confmat = self.confmat.compute().cpu().numpy()
        fig = self.plot_confusion_matrix(confmat)
        self.logger.experiment.add_figure("Confusion Matrix", fig, self.current_epoch)

        self.val_accuracy.reset()
        self.val_precision.reset()
        self.val_recall.reset()
        self.val_auroc.reset()
        self.val_f1.reset()  # Сброс F1 метрики
        self.confmat.reset()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

    def plot_confusion_matrix(self, confmat):
        fig, ax = plt.subplots(figsize=(8, 8))
        im = ax.imshow(confmat, interpolation='nearest', cmap=plt.cm.Blues)
        ax.figure.colorbar(im, ax=ax)
        ax.set(xticks=np.arange(confmat.shape[1]),
               yticks=np.arange(confmat.shape[0]),
               xticklabels=np.arange(10),
               yticklabels=np.arange(10),
               ylabel='True label',
               xlabel='Predicted label',
               title='Confusion Matrix')
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
        thresh = confmat.max() / 2.
        for i in range(confmat.shape[0]):
            for j in range(confmat.shape[1]):
                ax.text(j, i, format(confmat[i, j], 'd'),
                        ha="center", va="center",
                        color="white" if confmat[i, j] > thresh else "black")
        fig.tight_layout()
        return fig

# Подготовка данных
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = FashionMNIST(root=dataset_path, train=True, download=True, transform=transform)
val_dataset   = FashionMNIST(root=dataset_path, train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)



# Инициализируем модель
learn_id = str(uuid.uuid4())
model_name = 'fashion_MNIST_lite'
model = FashionMNISTLitModel(learning_rate=1e-3)
# Настраиваем TensorBoard логгер
logger = TensorBoardLogger(
        logs_path,
        name=model_name,
        version=learn_id,
        sub_dir=(_sd:=f"{(u_:=datetime.utcnow().strftime('%Y_%m_%d__%H_%M_%S'))}"),
)
# Настраиваем колбэк для сохранения модели
checkpoint_callback = ModelCheckpoint(
    monitor=(_tg_m:='val/auroc'),       # Метрика для мониторинга
    dirpath=test_weights_path, # Каталог для сохранения моделей
    filename=f'{model_name}_{_sd}_{learn_id}' + '-{epoch:02d}-{' + _tg_m + ':.2f}', # Имя файла
    save_top_k=1,             # Сохранять только лучшую модель
    mode='max'                # Минимизировать monitored метрику
)

# Создаем тренер PyTorch Lightning
trainer = pl.Trainer(
    max_epochs=20,
    logger=logger,
    log_every_n_steps=50,
    callbacks=[checkpoint_callback],
)
# Запускаем обучение
trainer.fit(model, train_loader, val_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /workspace/NN/my_checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type                      | Params | Mode 
---------------------------------------------------------------------
0 | model          | LightweightFashionMNIST   | 105 K  | train
1 | train_accuracy | MulticlassAccuracy        | 0      | train
2 | val_accuracy   | MulticlassAccuracy        | 0      | train
3 | val_precision  | MulticlassPrecision       | 0      | train
4 | val_recall     | MulticlassRecall          | 0      | train
5 | val_auroc      | MulticlassAUROC           | 0      | train
6 | confmat        | MulticlassConfusionMatrix | 0      | train
7 | val_f1         | MulticlassF1Score         | 0      | train
---------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


In [18]:
best_model_path = checkpoint_callback.best_model_path

target_path_to_model = os.path.join(prod_weights_path, os.path.split(os.path.split(best_model_path)[0])[1] + '_' + os.path.split(best_model_path)[1])

shutil.move(best_model_path, target_path_to_model)


FileNotFoundError: [Errno 2] No such file or directory: '/workspace/NN/my_checkpoints/fashion_MNIST_lite_2025_02_27__11_35_34_af5d8943-4cf8-4204-b7a3-f39e5a49e673-epoch=10-val/auroc=0.99.ckpt'

In [22]:
target_path_to_model = '/workspace/NN/neural/weights/prod/fashion_MNIST_lite_2025_02_27__11_35_34_af5d8943-4cf8-4204-b7a3-f39e5a49e673-epoch=10-val_auroc=0.99.ckpt'
trained_model = FashionMNISTLitModel.load_from_checkpoint(
    checkpoint_path=target_path_to_model,
    map_location=torch.device(device=device)  # или 'cuda' для GPU
)